In [1]:
from functions import *

In [2]:
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import glob

## Load data

### Choose inputs

In [3]:
source_dir = '../data/preprocessed'
data = ['lst','wt','ndvi']
all_ds = {k: [] for k in data}
for d in data:
    source_path = os.path.join(source_dir,d)
    for subdir, dirs, files in os.walk(source_path):
        for file in files:
            if file.endswith('.csv'):
                all_ds[d].append(pd.read_csv(f'{subdir}/{file}',index_col=0))
                

To choose specific rivers or not

In [7]:
filter_river = None
if filter_river != None:
    lst_array = all_ds['lst'][filter_river].values.flatten()
    ndvi_array = all_ds['ndvi'][filter_river].values.flatten()
    wt_array = all_ds['wt'][filter_river].values.flatten()
else:
    lst_array = np.concatenate([dataset.values.flatten() for dataset in all_ds['lst']])
    ndvi_array = np.concatenate([dataset.values.flatten() for dataset in all_ds['ndvi']])
    wt_array = np.concatenate([dataset.values.flatten() for dataset in all_ds['wt']])

In [8]:
use_ndvi = True
if use_ndvi:
  array_combined = np.stack((lst_array, ndvi_array), axis=-1)
  X = array_combined

else:
  X = lst_array
  X = X.reshape(-1, 1)

y = wt_array

In [9]:
X.shape

(83192, 2)

## Linear Regression model

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### K-fold cross validation

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = LinearRegression()

In [12]:
# Iterate over the folds
for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    total_results = evaluate_model(y_test, y_pred)


In [13]:
total_results

{'MAE': 1.4167301887166859,
 'MSE': 0.5715516392785962,
 'RMSE': 2.9336231001061974,
 'R²': 0.0008945378491006517,
 'MAPE (%)': 727629127252.3126,
 'MSE sample-wise': 8.606144493476696}

### Save model results

In [14]:
model_name = "pixel_wise_LR"
details = {'RMSE':total_results['RMSE'],'Variables':'lst, ndvi','Input': 'scaled 10 rivers', 'Output': 'wt', 'Resolution': 256, 'nº samples': len(y)}

file_path = f"../results/{model_name}_results.xlsx"
save_excel(file_path, details, excel = 'Results')

total_results['Model'] = model_name
file_path = f"../results/all_results.xlsx"
save_excel(file_path, total_results, excel = 'Results')